In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/project_folder

/content/gdrive/MyDrive/project_folder


In [ ]:
!pip install pmaw

     |████████████████████████████████| 176 kB 5.8 MB/s 
     |████████████████████████████████| 54 kB 2.6 MB/s 


In [ ]:
!pip install psaw

In [ ]:
!pip install config

In [ ]:
import pandas as pd
import numpy as np
import praw

In [ ]:
#reddit = praw.Reddit(
    #user_agent="name",
    #client_id="id",
    #client_secret="password"
    #username="USERNAME",
    #password="PASSWORD",
)

In [ ]:
from datetime import datetime

In [ ]:
from psaw import PushshiftAPI
#colllect posts
api = PushshiftAPI(reddit)

start_time = int(datetime(2020, 10, 1).timestamp())
end_time = int(datetime(2020, 12, 21).timestamp())

submissions = api.search_submissions(after=start_time,
                                     before = end_time,
                                     subreddit='wallstreetbets')
                                     #filter=['url','author', 'title', 'subreddit'])

In [ ]:
df = []
cnt = 0
for i in submissions:
  #print(vars(i))
  #if 'selftext' in vars(i):
  try:
    df.append({'title': i.title,
                'text': i.selftext,
                'url':i.url,
                'created_utc':i.created_utc,
                'author':i.author,
                'num_comments': i.num_comments,
                'score':i.score
                          })
  except AttributeError:
    continue
  cnt += 1  
  print(cnt)

In [ ]:
data = pd.DataFrame(df)

In [ ]:
data['time'] = data['created_utc'].apply(lambda x: datetime.fromtimestamp(x).strftime('%Y-%m-%d'))

In [ ]:
data.to_csv('reddit_2020.csv')

In [ ]:
#colllect comments

df_sub = []
df_comment = []
cnt = 0
for sub in submissions:
  cnt +=1
  print(cnt)
  #sub = reddit.submission(id=sub_id)
  try:
    df_sub.append({'id': sub.id,
                  'title': sub.title,
                  'text': sub.selftext,
                  'url':sub.url,
                  'created_utc':sub.created_utc,
                  'author':sub.author,
                  #'author_id':sub.author_fullname,
                  'num_comments': sub.num_comments,
                  'score':sub.score,
                  'likes':sub.likes,
                  'ups':sub.ups,
                  'downs':sub.downs
                            })
  except AttributeError:
      continue
  for comment in sub.comments.list():
    try:
      df_comment.append({'sub_id':sub.id,
                        'id':comment.id,
                        'text':comment.body,
                        'author':comment.author,
                        #'author_id':comment.author_fullname,
                        'created_utc':comment.created_utc,
                        'likes':comment.likes,
                        'score':comment.score,
                        'ups':comment.ups,
                        'downs':comment.downs,
                        'parent_id':comment.id})
    except AttributeError:
      continue

In [ ]:
pd.DataFrame(df_comment)

,sub_id,id,text,author,created_utc,likes,score,ups,downs,parent_id
0,jmcxo9,gaug6ma,/r/antstreetbets,diffcalculus,1.604275e+09,None,16,16,0,gaug6ma
1,jmcxo9,gaugezd,I’d probably invest it in a cheap hooker and s...,Order66JasonGenova,1.604275e+09,None,9,9,0,gaugezd
2,jmcxo9,gaugnif,Aww... that’s cute.,taddyjay,1.604275e+09,None,8,8,0,gaugnif
3,jmcxo9,gaugmcl,PS5,_JimN_,1.604275e+09,None,3,3,0,gaugmcl
4,jmcxo9,gauh3ht,Groceries,TacoFoods,1.604275e+09,None,3,3,0,gauh3ht
...,...,...,...,...,...,...,...,...,...,...
4572,jlswf2,gar5y70,Anagram,Hubers57,1.604196e+09,None,1,1,0,gar5y70
4573,jlswf2,garbmq2,Wow okay lots of great info! Thanks so much man!,l0ckituP,1.604200e+09,None,2,2,0,garbmq2
4574,jlswf2,gb5nuam,Did you abandon those vix debit spreads or get...,Wanderer1066,1.604524e+09,None,1,1,0,gb5nuam
4575,jlswf2,gb5pffg,"Worse. I took profit on those, flipped into sq...",l0ckituP,1.604525e+09,None,1,1,0,gb5pffg


In [ ]:
sub_df = pd.DataFrame(df_sub)
sub_df.to_csv("reddit_sub_1101.csv", encoding='utf-8', index=False)
comment_df = pd.DataFrame(df_comment)
comment_df.to_csv("reddit_comment_1101.csv", encoding='utf-8', index=False)
